In [3]:
import numpy as np
import random
import gurobipy as gp
from gurobipy import GRB


#This is the example in the paper "Identifying Minimally Infeasible Subsystems of Inequalities"
A=np.array([[1,-1],[0,2],[-1,-1],[0,-1],[-2,-1]])
b=np.array([[0],[1],[-2],[-2],[-4]])

M = np.r_[A.T, b.T,np.eye(len(A))]
c = np.zeros((sum(A.shape) + 1, 1))
c[A.shape[1]] = -1

Initial Constraint

$$Ax\leq b$$

The index of IIS is the support of the vertices of the polyhedron

$$A^T y=0$$
$$b^T y\leq -1$$
$$ y \geq 0$$

In [4]:
M

array([[ 1.,  0., -1.,  0., -2.],
       [-1.,  2., -1., -1., -1.],
       [ 0.,  1., -2., -2., -4.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])

In [5]:
c

array([[ 0.],
       [ 0.],
       [-1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [9]:
m=gp.Model()
x = m.addMVar ( shape=A.shape[0], lb=0, vtype =GRB.CONTINUOUS , name ="x")
m.addConstr (A.T @ x == 0 , name ="c")
m.addConstr (b.reshape(-1)@x ==-1, name='b')
w=np.array([1,1,1,1,1])
m.setObjective (w @ x, GRB. MAXIMIZE )
m.setParam('OutputFlag', 0)
for i in range(A.shape[0]):
    x.ub=np.inf*np.ones(A.shape[0])
    x[i].ub=0
    m.optimize()
    status = m.status
    if status == GRB.OPTIMAL:
        print('The optimal objective is %g')
        print(x.x)
    if status == GRB.INF_OR_UNBD or status == GRB.INFEASIBLE:
        print('Optimization was stopped with status %d' % status)
        pass

The optimal objective is %g
[0.         0.33333333 0.         0.66666667 0.        ]
Optimization was stopped with status 3
The optimal objective is %g
[0.8 0.6 0.  0.  0.4]
The optimal objective is %g
[1. 1. 1. 0. 0.]
The optimal objective is %g
[1. 1. 1. 0. 0.]


In [10]:
support = list(map(lambda x: x[1] if x[0]!=0 else None,zip(x.x,[0,1,2,3,4,5])))
support = list(filter(lambda x: x !=None , support))
support

[0, 1, 2]